In [ ]:
!unzip "/content/drive/MyDrive/IR/archive.zip"

unzip:  cannot find or open /content/drive/MyDrive/IR/archive.zip, /content/drive/MyDrive/IR/archive.zip.zip or /content/drive/MyDrive/IR/archive.zip.ZIP.


In [ ]:
# for loading/processing the images
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg19 import VGG19
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

In [ ]:
path = r"/content/Images"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
images = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the flowers list
            images.append(file.name)

In [ ]:
print(images[:10])

['2084157130_f288e492e4.jpg', '3616808182_fb4eaec778.jpg', '2067833088_04e84e5bf2.jpg', '2888408966_376c195b3f.jpg', '2902844125_4186bf3ab6.jpg', '2638369467_8fc251595b.jpg', '1130369873_d80a1aa59c.jpg', '3217893350_57be430d06.jpg', '3019857541_3de3e24f54.jpg', '3490528249_6aae9b867b.jpg']


In [ ]:
model = VGG19()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

574726144/574710816 [==============================] - 7s 0us/step


In [ ]:
features = {}
# lop through each image in the dataset
i = 0
for image in images:
    feat = extract_features(image,model)
    features[image] = feat
    if i%50==0:
      print(i)
    i=i+1

In [ ]:
afile = open(r'/content/features.pkl', 'wb')
pickle.dump(features, afile)

In [ ]:
with open(r"/content/drive/MyDrive/IR/features.pkl", "rb") as input_file:
    loaded_features = pickle.load(input_file)

In [ ]:
filenames = np.array(list(loaded_features.keys()))

In [ ]:
feat = np.array(list(loaded_features.values()))
feat.shape

(8091, 1, 4096)

In [ ]:
feat = feat.reshape(-1,4096)
feat.shape

(8091, 4096)

In [ ]:
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)


In [ ]:
with open("/content/drive/MyDrive/IR/pca.pkl", "wb") as f:
    pickle.dump(pca, f)

Clustering

In [ ]:
kmeans = KMeans(n_clusters=50,random_state=2)
kmeans.fit(x)

KMeans(n_clusters=50, random_state=2)

In [ ]:
with open("/content/drive/MyDrive/IR/kmeans.pkl", "wb") as f:
    pickle.dump(kmeans, f)

In [ ]:
kmeans.labels_[:30]

array([ 2, 11, 11, 14, 39,  7, 48, 33, 17, 43, 39,  4,  6, 13, 19, 34, 21,
       33, 41, 26,  4, 27,  4, 14, 48, 24, 42, 45,  4, 19], dtype=int32)

In [ ]:
with open("kmeans.pkl", "rb") as f:
    kmeans_model = pickle.load(f)


In [ ]:
from collections import Counter
c = Counter( kmeans.labels_ )

print( c.items() )


dict_items([(2, 159), (11, 195), (14, 240), (39, 239), (7, 199), (48, 157), (33, 242), (17, 109), (43, 128), (4, 320), (6, 118), (13, 32), (19, 226), (34, 188), (21, 238), (41, 160), (26, 141), (27, 134), (24, 229), (42, 119), (45, 162), (40, 315), (25, 198), (37, 85), (15, 193), (23, 193), (9, 114), (30, 268), (3, 158), (36, 206), (18, 150), (47, 97), (5, 130), (8, 240), (22, 71), (20, 160), (28, 140), (10, 119), (38, 153), (32, 141), (49, 172), (35, 171), (31, 98), (12, 104), (1, 239), (0, 73), (46, 105), (29, 92), (16, 121), (44, 50)])


In [ ]:
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
groups[2]

In [ ]:
afile = open(r'/content/cluster.pkl', 'wb')
pickle.dump(groups, afile)
afile.close()